데이터 로드 & 기본 구조 파악

In [1]:
import numpy as np
import pandas as pd

# CSV Load
path = "data/IMDB_Dataset.csv"
df = pd.read_csv(path)

# 기본 구조 파악
display(df.head(3))

print(df.info(), '\n') # row/col 수, 데이터 타입, 결측치 여부 확인
print(df.shape, '\n')
print(df.describe(include="all"), '\n') # count/unique/top(최빈값)/freq(최빈값 빈도수)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None 

(50000, 2) 

                                                   review sentiment
count                                               50000     50000
unique                                              49582         2
top     Loved today's show!!! It was a variety and not...  positive
freq                                                    5     25000 



컬럼 전처리

In [3]:
# 컬럼 표준화
df.columns = [c.lower() for c in df.columns]
# 꼭 있어야 할 컬럼들이 실제 데이터프레임에 있는지 확인 (없으면 경고)
assert set(["review", "sentiment"]).issubset(df.columns)

# 레이블 매핑 : sentiment -> 숫자 (컬럼 추가)
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})
print(df["label"].value_counts(dropna=False)) # 결측치 포함 카운트 
# => 긍정:부정이 1:1로 데이터셋이 균형잡혀있음을 확인

label
1    25000
0    25000
Name: count, dtype: int64


In [4]:
# 특징 컬럼 추가 : 리뷰 길이, 단어 수
df["len_chars"] = df["review"].str.len()
df["num_tokens"] = df["review"].str.split().apply(len)

display(df[["review", "sentiment", "label", "len_chars", "num_tokens"]].head(3))

,review,sentiment,label,len_chars,num_tokens
0,One of the other reviewers has mentioned that ...,positive,1,1761,307
1,A wonderful little production. <br /><br />The...,positive,1,998,162
2,I thought this was a wonderful way to spend ti...,positive,1,926,166


결측치/이상치 처리 : 학습 전 에러 방지 위한 클리닝

In [5]:
# 결측치 확인
null_report = df.isnull().sum().sort_values(ascending=False) # 결측치 많은 순
print(null_report, '\n')

review        0
sentiment     0
label         0
len_chars     0
num_tokens    0
dtype: int64 



In [6]:
# 결측치 처리
# 전략 : review 결측은 빈 문자열 대체 / label 결측은 학습 불가하므로 해당 행 제거
df["review"] = df["review"].fillna("")
df = df.dropna(subset=["label"]).copy()
# Pandas에서 기존 DataFram으로 슬라이싱/필터링하면 SettingWithCopyWarning
# .copy()로 복사본 만들어 안전하게 사용

In [8]:
# 공백/이상치 처리 : 토큰 길이 0인 리뷰 제거
df = df[df["num_tokens"] > 0].copy()
print("After Cleaning: ", df.shape, '\n')

After Cleaning:  (50000, 5) 



레이블 별 수치 통계 ⇒ EDA

In [12]:
# 레이블(긍정/부정)별 그룹 통계
by_label = (
	df.groupby("label")[["len_chars", "num_tokens"]]
		.agg(["mean", "std", "min", "max", "median"])
)
display(by_label)

# descibe로도 확인 가능
display(df.groupby("label")["num_tokens"].describe())

# 클래스 불균형 점검(%) 
print(df["label"].value_counts(normalize=True).rename("ratio"))

len_chars                                num_tokens                  \
             mean          std min    max median       mean         std min   
label                                                                         
0      1294.06436   945.892669  32   8969  973.0  229.46456  164.947795   4   
1      1324.79768  1031.492627  65  13704  968.0  232.84932  177.497046  10   

                    
        max median  
label               
0      1522  174.0  
1      2470  172.0

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,25000.0,229.46456,164.947795,4.0,128.0,174.0,278.0,1522.0
1,25000.0,232.84932,177.497046,10.0,125.0,172.0,284.0,2470.0


label
1    0.5
0    0.5
Name: ratio, dtype: float64


사용자 메타 + 행동 로그 결합(merge) 구조
- 추천 시스템 데이터의 전형적 스키마 : 유저 - 아이템(평점)
- 실습 → 샘플링으로 가상(가짜) user_id, item_id 만들어서 사용

In [14]:
# 샘플 수 설정(5 ~ 10k 권장)
N = min(8000, len(df))

# 샘플링(인덱스 재부여)
sample = df.iloc[:N].copy().reset_index(drop=True)

# 가상 item_id 생성 (지금의 item은 리뷰)
sample["item_id"] = np.arange(1, N+1)

# 가상 사용자 풀 생성
# np.random.seed(4) # 랜덤성 통제
num_users = 1000
users = pd.DataFrame({
	"user_id": np.arange(1, num_users+1),
	"age_group": np.random.choice(["18-24", "25-34", "35-44", "45+"], size=num_users, p=[0.35, 0.4, 0.2, 0.05]),
	"region": np.random.choice(["Seoul", "Busan", "Daegu", "Daejeon", "Incheon", "Gwangju"], size=num_users)
})

# sample에 user_id 할당 (RR)
sample["user_id"] = (sample.index % num_users) + 1

# 평점 구성 : sentiment(0/1)을 1~5 스케일로 매핑
# 부정(0)은 1~2 / 긍정(1)은 4~5, 무작위 선택
pos_mask = sample["label"] == 1
neg_mask = ~pos_mask
sample.loc[pos_mask, "rating"] = np.random.choice([4,5], size=pos_mask.sum(), p=[0.5,0.5])
sample.loc[neg_mask, "rating"] = np.random.choice([1,2], size=neg_mask.sum(), p=[0.5,0.5])
sample["rating"] = sample["rating"].astype(int)

# 사용자 정보와 merge (key: user_id)
interactions = pd.merge(
	sample[["user_id", "item_id", "rating", "review", "label", "num_tokens"]],
	users,
	on="user_id",
	how="left"
)

display(interactions.head(3))
print(interactions.info(), '\n')

,user_id,item_id,rating,review,label,num_tokens,age_group,region
0,1,1,4,One of the other reviewers has mentioned that ...,1,307,25-34,Daejeon
1,2,2,5,A wonderful little production. <br /><br />The...,1,162,25-34,Seoul
2,3,3,5,I thought this was a wonderful way to spend ti...,1,166,35-44,Incheon


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     8000 non-null   int64 
 1   item_id     8000 non-null   int64 
 2   rating      8000 non-null   int64 
 3   review      8000 non-null   object
 4   label       8000 non-null   int64 
 5   num_tokens  8000 non-null   int64 
 6   age_group   8000 non-null   object
 7   region      8000 non-null   object
dtypes: int64(5), object(3)
memory usage: 500.1+ KB
None 



추천용 피봇 테이블 : 사용자 x 아이템(평점) 행렬 생성

In [15]:
# 사용자 x 평점 매트릭스 (N이 크면 메모리 부담 있으므로 샘플 축소)
M = min(2000, len(interactions))
ratings_small = interactions.iloc[:M]

user_item = ratings_small.pivot_table(
	index = "user_id",
	columns = "item_id",
	values = "rating",
	aggfunc = "mean" # 중복 시 평균
)

display(user_item.iloc[:5, :10])
print("Num of users:", user_item.shape[0], "Num of items:", user_item.shape[1])

# Sparsity : 전체 개수 분의 NaN 아닌 값의 개수. 분모=0을 방지하기 위해 분모에 아주 작은 값, 1e-9을 더함
print("Sparsity(%) ~", round(100*(1 - user_item.count().sum()/(user_item.shape[0]*user_item.shape[1] + 1e-9)), 2))

item_id,1,2,3,4,5,6,7,8,9,10
user_id,,,,,,,,,,
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN


Num of users: 1000 Num of items: 2000
Sparsity(%) ~ 99.9


산출물 저장 : 재현성 & 다음 단계(ML/DL)로 바로 넘길 수 있게

In [18]:
# 산출물 저장

# 1) 전처리된 원본 일부
df_clean = df[["review", "label", "len_chars", "num_tokens"]].copy()
df_clean.to_csv("data/imdb_clean.csv", index=False, encoding="utf-8")

# 2) 사용자 메타
users.to_csv("data/users_mock.csv", index=False, encoding="utf-8")

# 3) 상호작용 로그
interactions[["user_id", "item_id", "rating", "label", "num_tokens"]].to_csv(
	"data/interactions_mock.csv", index=False, encoding="utf-8"
)

# 4) 사용자-평점 매트릭스
user_item.to_csv("data/user_item_matrix.csv", encoding="utf-8")

print("Saved: imdb_clean.csv, users_mock.csv, interactions_mock.csv, user_item_matrix.csv")

Saved: imdb_clean.csv, users_mock.csv, interactions_mock.csv, user_item_matrix.csv
